<a href="https://colab.research.google.com/github/Nehach73/DS3001_project_voting/blob/main/Project_Notebook_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project: Voting

**Group 2: Randa Ampah, Neha Channamraju, Rebekah Weaver, Faith Koobial, Gian Baez, Aysha Hussen**

In [46]:
# Importing the necessary libraries
import geopandas as gpd
!pip install mapclassify
import mapclassify
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [58]:
# Loading in the data
! git clone https://github.com/Nehach73/DS3001_project_voting.git

voting = pd.read_csv('DS3001_project_voting/data/voting_VA.csv')
voting = voting.drop(columns=['Unnamed: 0', 'state', 'state_po', 'office', 'version', 'mode'], axis=1)
voting.head()

fatal: destination path 'DS3001_project_voting' already exists and is not an empty directory.


,year,county_name,county_fips,candidate,party,candidatevotes,totalvotes
0,2000,ACCOMACK,51001,AL GORE,DEMOCRAT,5092,11925
1,2000,ACCOMACK,51001,GEORGE W. BUSH,REPUBLICAN,6352,11925
2,2000,ACCOMACK,51001,RALPH NADER,GREEN,220,11925
3,2000,ACCOMACK,51001,OTHER,OTHER,261,11925
4,2000,ALBEMARLE,51003,AL GORE,DEMOCRAT,16255,36846


In [59]:
counties = pd.read_csv('DS3001_project_voting/data/county_adjacencies.csv')
counties = counties.rename(columns={'County':'county_name', 'FIPS':'county_fips'})
counties['county_name'] = counties['county_name'].str.upper()
counties.head()

,county_name,Population2022,county_fips,District,N1,N2,N3,N4,N5,N6,N7,N8,N9,N10,N11,N12
0,ACCOMACK,33191,51001,2,Northumberland,Lancaster,Middlesex,Mathews,Northampton,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALBEMARLE,114534,51003,5,CharlottesvilleCity,Nelson,Augusta,Rockingham,Greene,Orange,Louisa,Fluvanna,Buckingham,Nelson,NaN,NaN
2,ALEXANDRIACITY,155525,51510,8,Fairfax,Arlington,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ALLEGHANY,14835,51005,6,Craig,Bath,Botetourt,CovingtonCity,Rockbridge,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AMELIA,13455,51007,5,PrinceEdward,Cumberland,Powhatan,Chesterfield,Dinwiddie,Nottoway,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
# Combining both datasets
voting = pd.merge(voting, counties, on=['county_name','county_fips'], how='left')
voting.head()

,year,county_name,county_fips,candidate,party,candidatevotes,totalvotes,Population2022,District,N1,...,N3,N4,N5,N6,N7,N8,N9,N10,N11,N12
0,2000,ACCOMACK,51001,AL GORE,DEMOCRAT,5092,11925,33191.0,2.0,Northumberland,...,Middlesex,Mathews,Northampton,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,ACCOMACK,51001,GEORGE W. BUSH,REPUBLICAN,6352,11925,33191.0,2.0,Northumberland,...,Middlesex,Mathews,Northampton,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,ACCOMACK,51001,RALPH NADER,GREEN,220,11925,33191.0,2.0,Northumberland,...,Middlesex,Mathews,Northampton,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,ACCOMACK,51001,OTHER,OTHER,261,11925,33191.0,2.0,Northumberland,...,Middlesex,Mathews,Northampton,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,ALBEMARLE,51003,AL GORE,DEMOCRAT,16255,36846,114534.0,5.0,CharlottesvilleCity,...,Augusta,Rockingham,Greene,Orange,Louisa,Fluvanna,Buckingham,Nelson,NaN,NaN
